In [2]:
import ml
import nlp
import json_io
import pickle
from itertools import chain
from dvs import DictVectorizerPartial
import numpy as np

### Process comment

In [ ]:
path = ml.JSON_DIR+"reddit/"
sarcastic_path = path+"sarcastic/"
serious_path = path+"serious/"
source = '-reddit-'
features_path = 'features/'
n=10

In [ ]:
json_io.processRandomizeJson(sarcastic=True,
                     json_path=sarcastic_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensReddit)
json_io.processRandomizeJson(sarcastic=False,
                     json_path=serious_path,
                     features_path=features_path,
                     source=source,
                     n=n,
                     cleanTokens=nlp.cleanTokensReddit)

### Load set of features

In [ ]:
sarcasticFeats = json_io.loadProcessedFeatures(features_path,
                                       source,
                                       sarcastic=True,
                                       n=5,
                                       random=False)
seriousFeats = json_io.loadProcessedFeatures(features_path,
                                     source,
                                     sarcastic=False,
                                     n=3,
                                     random=False)
features = chain(sarcasticFeats, seriousFeats)

### Flatten feature dictionaries, if leaveout is a feature that feature is ommitted

In [ ]:
dvp = DictVectorizerPartial()

In [ ]:
(X,y) = ml.split_feat(features, 2)

In [ ]:
(X,y) = ml.flatten(X,y)

In [ ]:
(X,y) = (dvp.partial_fit_transform(X), np.array(list(y)))

In [ ]:
pickle.dump(dvp, open('pickled/-reddit-dvp.pickle', 'wb'))
pickle.dump(y, open('pickled/-reddit-y.pickle', 'wb'))
pickle.dump(X, open('pickled/-reddit-X.pickle', 'wb'))

In [3]:
X = pickle.load(open('pickled/-reddit-X.pickle', 'rb'))
y = pickle.load(open('pickled/-reddit-y.pickle', 'rb'))

### Train and test, reports results

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
classifiers = [
    #LogisticRegression(n_jobs=-1),
    #SGDClassifier(loss='log'),
    #BernoulliNB(),
    MultinomialNB(),
]

In [ ]:
results = []
for reduceamount in [0, 2500000, 1000000, 500000, 100000]: #, 50000, 25000, 10000, 7500, 5000, 2500, 1500, 1000, 750, 500, 250, 100, 50, 10, 5]:
    print("\n\t\tReduction: "+str(reduceamount))
    for trainsize in [0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8]:
        print("\n\t\tTraining size: "+str(trainsize))
        results.append((reduceamount,
                       trainsize,
                       ml.trainTest(X,
                                    y,
                                    classifiers=classifiers,
                                    reduce=reduceamount,
                                    splits=2,
                                    trainsize=trainsize,
                                    testsize=0.2)))
pickle.dump(results, open('pickled/-reddit-trained.pickle', 'wb'))
print(results)    


		Reduction: 0

		Training size: 0.01
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.653264
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.643739

		Training size: 0.05
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.688723
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.685449

		Training size: 0.1
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 1	Score:	0.700569
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 1	Score:	0.704171

		Training size: 0.2
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 1	

C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:113: UserWarning: Features [0 0 0 ..., 0 0 0] are constant.
  UserWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Features after reduction: (7453, 2500000)
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.651829
Features before reduction: (7453, 12490143)
Features after reduction: (7453, 2500000)
Starting to train <class 'sklearn.naive_bayes.MultinomialNB'>
<class 'sklearn.naive_bayes.MultinomialNB'>	Time: 0	Score:	0.620898

		Training size: 0.05


In [ ]:
xyz = []
for red, train, res in results:
    acc = [r[2] for r in res]
    xyz.append((red, train, sum(acc)/len(acc)))
json_io.list_to_json(xyz, "-reddit-reduction-trainsize-accuracy-mnbayes.json", old_format=True)

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
classifiers = [
    LogisticRegression(n_jobs=-1),
    #SGDClassifier(loss='log'),
    #BernoulliNB(),
    #MultinomialNB(),
]
results = []
for reduceamount in [0, 2500000, 1000000]: #, 50000, 25000, 10000, 7500, 5000, 2500, 1500, 1000, 750, 500, 250, 100, 50, 10, 5]:
    print("\n\t\tReduction: "+str(reduceamount))
    for trainsize in [0.01, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8]:
        print("\n\t\tTraining size: "+str(trainsize))
        results.append((reduceamount,
                       trainsize,
                       ml.trainTest(X,
                                    y,
                                    classifiers=classifiers,
                                    reduce=reduceamount,
                                    splits=2,
                                    trainsize=trainsize,
                                    testsize=0.2)))
pickle.dump(results, open('pickled/-reddit-trained-log.pickle', 'wb'))
print(results)    

In [ ]:
xyz = []
for red, train, res in results:
    acc = [r[2] for r in res]
    xyz.append((red, train, sum(acc)/len(acc)))
json_io.list_to_json(xyz, "-reddit-reduction-trainsize-accuracy-log.json", old_format=True)